# Bestimmung von Textlängen

Wenn du viele Dokumente analysieren möchtest, spielt deren individuelle Länge eine große Rolle. Das gilt nicht nur für die Größe in Bytes, sondern auch die Anzahl der Zeichen (das ist *nicht* identisch z.B. wegen der UTF-8-Kodierung), die Anzahl der Tokens, der Sätze usw.

`textacy` und `spacy` bieten hier eine gute Kombination von Auswertungsmöglichkeiten.

## Alle Dokumente verarbeiten

Für dieses Beispiel wirst du nun alle Dokumente analysieren lassen, dazu wählst du alle aus der Datenbank aus:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM articles ORDER BY datePublished", sql)

Da dich der gesamte Text interessiert, fügst du die einzelnen Textfelder zusammen:

In [ ]:
df["full_text"] = df["title"] + "\n" + df["header"] + "\n" + df["text"]

Für die Textanalyse benötigt `textacy` die Hilfe von `spacy`, das du als Modul importierst und das Modell lädst (diesmal aber nur mit dem `parser`, damit die Analyse etwas schneller funktioniert):

In [ ]:
!pip install textacy
!python -m spacy download de_core_news_lg

In [ ]:
import spacy

nlp = spacy.load("de_core_news_lg", disable=("parser",))

Die Analyse dauert dennoch ein paar Minuten:

In [ ]:
import textacy

corpus = textacy.Corpus(nlp, data = list(df["full_text"].values))

Jetzt kannst du auf statistische Informationen des gesamtem Corpus komfortabel zugreifen:

In [ ]:
corpus.n_docs, corpus.n_sents, corpus.n_tokens

Die Anzahl der Tokens entspricht der Länge des Dokuments und du kannst eine eigene Spalte im `DataFrame` belegen:

In [ ]:
df["tokens"] = [len(d) for d in corpus]

Die Anzahl der Buchstaben kannst du auch leicht ermitteln (allerdings sind darin Leerzeichen und Umbrüche auch enthalten):

In [ ]:
df["characters"] = df["full_text"].str.len()

Mit `describe` kannst du dir statistiscge Informationen zu den Spalten ausgeben lassen:

In [ ]:
df.describe()

Daran erkennst du schon einiges an der Datenstruktur, allerdings kannst du das besser visuell erkunden. Dazu verwendest du *Histpgramme*, die dir die Anzahl der Dokumente in einzelnen Wertebereichen anzeigen. Betrachte zuerst die Anzahl der Tokens:

In [ ]:
!pip install matplotlib

In [ ]:
df[["tokens"]].plot.hist()

Wenn du eine genauere Auflösung brauchst, kann du mehr sog. *Bins* verwenden:

In [ ]:
df[["tokens"]].plot.hist(bins=40)

Ähnlich wie in der *Five-Number-Summary* oben kannst du hier gut erkennen, dass die meisten Artikel zwischen 300 und 500 Tokens haben. Es gibt wenige, die deutlich länger sind.

Du kannst dir auch die Anzahl der Buchstaben als Histogramm ausgeben lassen:

In [ ]:
df[["characters"]].plot.hist(bins=40, logy=True)

Auch damit ergibt sich die typische Kurve, die du bereits bei der Anzahl der Tokens gesehen hast. Selbstverständlich kannst du die Auswertungen miteinander kombinieren, um z.B. die typische Länge der Tokens in Zeichen zu ermitteln usw.

## Textlängen nicht unterschätzen

In den nächsten Auswertungen wirst du dich noch häufiger mit Textlängen auseinandersetzen. Besonders im Zeitverlauf ist es neben der Anzahl der Texte ein entscheidendes Kriterium, wie aussagekräftig ein Korpus ist. 